In [1]:
import numpy
import theano
import theano.tensor as T
import six.moves.cPickle as pickle
import matplotlib.pyplot as plt
%matplotlib inline

Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 10.0% of memory, cuDNN 5004)
//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
class LogisticRegression(object):
    def __init__(self, input, n_in, n_out):
        self.W = theano.shared(
            value = numpy.zeros((n_in, n_out), dtype = theano.config.floatX),
            name = "W",
            borrow = True
        )
        self.b = theano.shared(
            value = numpy.zeros((n_out, ), dtype = theano.config.floatX),
            name = "b",
            borrow = True
        )
        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W) + self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis = 1)
        self.params = [self.W, self.b]
        self.input = input
    
    def negloglik(self, y, lamb):
        return (-T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y]) +\
                lamb * T.sum(abs(self.W)))
    
    def errors(self, y):
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()

In [3]:
def load_data():
    with open('/Users/tnybny/Documents/Kaggle/Digit recognizer/Data/train', 'r') as f:
        D = pickle.load(f)[1:, ]
        indices = numpy.random.permutation(D.shape[0])
        tr_idx, va_idx = indices[:28000], indices[28001:]
        train_set = (D[tr_idx, 1:], D[tr_idx, 0])
        valid_set = (D[va_idx, 1:], D[va_idx, 0])
    with open('/Users/tnybny/Documents/Kaggle/Digit recognizer/Data/test', 'r') as f:
        Dtest = pickle.load(f)[1:, ]
        test_set = (Dtest, Dtest[:, 0])

    def shared_dataset(data_xy, borrow = True):
        data_x, data_y = data_xy
        shared_x = theano.shared(numpy.asarray(data_x,
                                              dtype = theano.config.floatX),
                                borrow = borrow)
        shared_y = theano.shared(numpy.asarray(data_y,
                                              dtype = theano.config.floatX),
                                borrow = borrow)
        return shared_x, T.cast(shared_y, 'int32')
    
    Xtrain, ytrain = shared_dataset(train_set)
    Xval, yval = shared_dataset(valid_set)
    Xtest, ytest = shared_dataset(test_set)
    rval = [(Xtrain, ytrain), (Xval, yval), (Xtest, ytest)]
    return rval

In [4]:
def sgd_optimization_mnist(D, alpha = 0.001, lamb = 0.1, n_epochs = 100, batch_size = 100):
    Xtrain, ytrain = D[0]
    Xval, yval = D[1]
    Xtest, ytest = D[2]

    n_train_batches = Xtrain.get_value(borrow = True).shape[0] // batch_size
    n_valid_batches = Xval.get_value(borrow = True).shape[0] // batch_size
    
    index = T.lscalar() # mini-batch index
    x = T.matrix('x') # symbols for inputs and outputs
    y = T.ivector('y')
    
    classifier = LogisticRegression(input = x, n_in = 28 * 28, n_out = 10)
    cost = classifier.negloglik(y, lamb)
    
    test_model = theano.function(
        inputs = [index],
        outputs = classifier.errors(y),
        givens = {
            x: Xtest[index * batch_size: (index + 1) * batch_size],
            y: ytest[index * batch_size: (index + 1) * batch_size]
        }
    )
    
    valid_model = theano.function(
        inputs = [index],
        outputs = classifier.errors(y),
        givens = {
            x: Xval[index * batch_size: (index + 1) * batch_size],
            y: yval[index * batch_size: (index + 1) * batch_size]
        }
    )
    
    g_W = T.grad(cost = cost, wrt = classifier.W)
    g_b = T.grad(cost = cost, wrt = classifier.b)
    
    updates = [(classifier.W, classifier.W - alpha * g_W),
               (classifier.b, classifier.b - alpha * g_b)]
    
    train_model = theano.function(
        inputs = [index],
        outputs = classifier.errors(y),
        updates = updates,
        givens = {
            x: Xtrain[index * batch_size: (index + 1) * batch_size],
            y: ytrain[index * batch_size: (index + 1) * batch_size]
        }
    )
    
    improvement_threshold = 0.95
    validation_frequency = n_train_batches
    best_valid_loss = numpy.inf
    best_minibatch_loss = numpy.inf
    test_score = 0.
    epoch = 0
    costs = []
    valid_costs = []
    train_costs = []
    while (epoch < n_epochs):
        epoch = epoch + 1
        
        if epoch % 5 == 0:
                    alpha = alpha / 2
        
        for minibatch_index in range(n_train_batches):
            minibatch_avg_cost = train_model(minibatch_index)
            train_costs.append(minibatch_avg_cost)
            
            iter = (epoch - 1) * n_train_batches + minibatch_index # count iterations
            if (iter + 1) % validation_frequency == 0:
                    
                valid_losses = [valid_model(i) for i in range(n_valid_batches)]
                this_valid_loss = numpy.mean(valid_losses)
                valid_costs.append(this_valid_loss)
                costs.append(numpy.mean(train_costs))
                train_costs = []
                
                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_valid_loss * 100.
                    )
                )
                    
                if this_valid_loss < best_valid_loss:
                    best_valid_loss = this_valid_loss

                    with open('LogReg/best_model.pkl', 'wb') as f:
                        pickle.dump(classifier, f)
    print(
        (
            'Optimization complete with best validation score of %f %%,'
            'with test performance %f %%'
        )
        % (best_valid_loss * 100., test_score * 100.)
    )
    print('The code run for %d epochs') % (epoch)
    return costs, valid_costs

In [5]:
def predict():
    classifier = pickle.load(open('LogReg/best_model.pkl'))
    
    predict_model = theano.function(
        inputs = [classifier.input],
        outputs = classifier.y_pred)
    
    D = load_data()
    Xtest, ytest = D[2]
    Xtest = Xtest.get_value()

    predicted_values = predict_model(Xtest)
    numpy.savetxt('LogReg/submission.csv',
                  numpy.c_[range(1, Xtest.shape[0] + 1), predicted_values],
                  delimiter = ',', header = 'ImageId,Label', comments = '', fmt = '%d')

In [ ]:
D = load_data()
costs = []
valid_costs = []
alphas = numpy.random.uniform(0.0001, 0.1, 5).tolist() # random search is better than a grid search
lambdas = numpy.random.uniform(0.01, 10, 5).tolist() # over hyperparameter space
cost, valid_cost = sgd_optimization_mnist(D, 0.000002, 0.0001)

In [ ]:
plt.plot(cost)

In [ ]:
plt.plot(valid_cost)

In [6]:
predict()